# Instalaciones preliminares
*Antes de comenzar estas lineas de código debe instalar las ultimas versiones de los paquetes*
- Scikit-learn: Buscando en anaconda scikit-learn o poniendo en el terminal conda install scikit-learn o pip install scikit-learn ---> Más info en https://scikit-learn.org/stable/install.html
- Pandas: Buscando en anaconda pandas o poniendo en el terminal conda install pandas o pip install pandas ---> Más info en https://pandas.pydata.org/docs/getting_started/index.html
- Numpy: Buscando en anaconda numpy o poniendo en el terminal conda install numpy o pip install numpy ---> Más info en https://numpy.org/install/
- Imbalanced-learn: Poniendo en el terminal conda install -c conda-forge imbalanced-learn o pip install -U imbalanced-learn ---> Más info en https://imbalanced-learn.readthedocs.io/en/stable/install.html
- Pickle: Poniendo en el terminal pip install pickle ---> Más info si no te funciona en https://stackoverflow.com/questions/48477949/not-able-to-pip-install-pickle-in-python-3-6/48477988
- seaborn: Buscando en anaconda seaborn o poniendo en el terminal pip install seaborn o conda install seaborn ---> Más info en https://seaborn.pydata.org/installing.html
- matplotlib: Buscando en anaconda matplotlib o poniendo en el terminal pip install -U matplotlib ---> Más info en https://matplotlib.org/users/installing.html
- Keras: Buscando e instalando en anaconda los módulos de tensorflow y keras. También se pueden instalar usando los comandos pip install tensorflow y pip install keras ---> Más infor en https://keras.io/about/

Base de datos obtenida de: https://www.kaggle.com/mlg-ulb/creditcardfraud

# Lectura y balanceo de datos

## Lectura

In [1]:
import csv
import numpy as np

#Nombre del fichero que quiero leer
datos = "creditcard.csv"

#Inicialización de los arrays
caracteristicas = []
clase = []

#Lectura de datos
with open(datos) as f:
    for i, linea in enumerate(f):
        #Saltamos la cabecera del csv
        if i == 0:
            print("Cabecera:", linea.strip())
            continue  
        #Introducimos los datos en los arrays de características
        campos = linea.strip().split(",")
        caracteristicas.append([float(v.replace('"', "")) for v in campos[:-1]])
        clase.append([int(campos[-1].replace('"', ""))])
        if i == 1:
            print("Ejemplo de caracteristicas:", caracteristicas[-1])

#Conversión de datos
data = np.array(caracteristicas, dtype="float32")
target = np.array(clase, dtype="uint8")

#Pintamos la forma de los datos
print("Forma de los datos de entrada al modelo:", data.shape)
print("Forma de las clases de salida:", target.shape)

Cabecera: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
Ejemplo de caracteristicas: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
Forma de los datos de entrada al modelo: (284807, 30)
Forma de las clases de salida: (284807, 1)


## Balanceo 

In [2]:
#Primero analizamos los datos

#Inicializacion de la cuenta
legal = 0
fraude = 0

#Cuenta de datos
for x in range(target.shape[0]):
    if target[x] == 0:
        legal = legal + 1
    else:
        fraude = fraude + 1

#Representación
print("Tarjetas legales: " + str(legal))
print("Tarjetas fraudulentas: " + str(fraude))

Tarjetas legales: 284315
Tarjetas fraudulentas: 492


In [3]:
#Sobremuestreamos por la gran diferencia

#Importamos los paquetes de sobremuestreo
from imblearn.over_sampling import SMOTE

#SMOTE
smote = SMOTE()

#Generación de nuevas muestras sintéticas
dataSmote, targetSmote = smote.fit_resample(data,target)

#Volvemos a contar
legal = 0
fraude = 0

#Cuenta de datos
for x in range(targetSmote.shape[0]):
    if targetSmote[x] == 0:
        legal = legal + 1
    else:
        fraude = fraude + 1

#Representación
print("Tarjetas legales balanceadas: " + str(legal))
print("Tarjetas fraudulentas balanceadas: " + str(fraude))

Tarjetas legales balanceadas: 284315
Tarjetas fraudulentas balanceadas: 284315


In [4]:
#División de datos en conjunto de evaluación y conjunto de entrenamiento
from sklearn.model_selection import train_test_split
dataTrain, dataTest, targetTrain, targetTest = train_test_split(dataSmote,targetSmote, random_state = 0)

In [5]:
#Normalización de los datos 

#Cálculo de la media
mean = np.mean(dataTrain, axis=0)

#Restamos a las características la media
dataTrain -= mean
dataTest -= mean

#Cálculo de la desviación estándar
std = np.std(dataTrain, axis=0)

#Dividimos entre la desviación estándar
dataTrain /= std
dataTest /= std

# Entrenamiento y evaluación de modelos

## Construcción del modelo con Keras

In [6]:
from tensorflow import keras

#Añadimos las capas de nuestra red neuronal (3 densas y dos de dropout)
#Otra opcion seria añadirlas con add (próximo video)
model = keras.Sequential(
    [
        #Capa densa, la primera capa siempre tiene que especificar la forma de entrada
        keras.layers.Dense(
            256, activation="relu", input_shape=(dataTrain.shape[-1],) #Nodos de la capa densa, y función de activación
        ),
        #keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(25, activation="relu"),
        #Capa de Dropout. Inactiva algunos de los nodos de la red para evitar el sobreentrenamiento
        keras.layers.Dropout(0.3), #El atributo que se pone es el ratio de inactivación
        #keras.layers.Dense(256, activation="relu"),
        keras.layers.Dense(25, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)

#Vemos la forma de nuestro modelo
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 25)                6425      
_________________________________________________________________
dropout (Dropout)            (None, 25)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)                650       
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 15,037
Trainable params: 15,037
Non-trainable params: 0
____________________________________________________

## Entrenamiento y evaluación del modelo

In [7]:
#Funciones de las métricas (sacadas de internet: https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model)
from keras import backend as K

#Funciones de las métricas
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [8]:
#Compilamos el modelo
model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=['accuracy',f1_m,precision_m, recall_m]
) #Compilamos el modelo con su optimizador, la forma en la que actualizamos los pesos (minimización) y en base a qué métricas

#Lo entrenamos con los datos de entrenamiento
model.fit(
    dataTrain,
    targetTrain,
    #batch_size=2048,
    batch_size=20,
    epochs=10,
    verbose=1,
    validation_data=(dataTest, targetTest),
)#Conjuntos de entrenamientos y evaluación, numero de muestras en la propagación hacia atrás, 
#numero de iteraciones para mejorar el modelo, la verbosidad y los conjuntos de validación

Epoch 1/10
21324/21324 [==============================] - 74s 3ms/step - loss: 0.0626 - accuracy: 0.9858 - f1_m: 0.9850 - precision_m: 0.9839 - recall_m: 0.9877 - val_loss: 0.1520 - val_accuracy: 0.9834 - val_f1_m: 0.9822 - val_precision_m: 0.9959 - val_recall_m: 0.9707s: 0.1361 - accuracy: 0.9566 - f1_m: 0.9542 - precision_m: 0.9718 - recall_m: 0.94 - ETA: 51s - loss: 0.1326 - accuracy: 0.9573 - f1_m: 0.9545 - precision_m: 0 - ETA: 47s - loss: 0.1156 - accuracy: 0.9631 - f1_m: 0.9609 - precision_m: 0.9758 - recall_m: 0.95 - ETA: 47s - loss: 0.1183 - accuracy: 0.9629 - f1_m: 0.9606 - precision_m: 0.9757 - recall_m - ETA: 47s - loss: 0.1378 - accuracy: 0.9627 - f1_m: 0.9605 - precision_m: 0.9749 - recall_m - ETA: 46s - loss: 0.1320 - accuracy: 0.9641 - f1_m: 0.9620 - pr - ETA: 44s - loss: 0.1136 - accuracy: 0.9687 - f - ETA: 41s - loss: 0.0981 - accuracy: 0.9722 - f1_m: 0.9706 - precision_m: 0.9781 - recall_m: - ETA: 40s - loss: 0.0963 - accuracy: 0.9727 - f1_m: 0.9712 - precision_m: 0.

In [9]:
#Métricas
#Conjunto de entrenamiento
print("Datos sobre el entrenamiento")
loss, accuracy,f1_score, precision, recall = model.evaluate(dataTrain, targetTrain, verbose=False)
print("Exactitud de entrenamiento: {:.4f}".format(accuracy))
print("F1 de entrenamiento: {:.4f}".format(f1_score))
print("Precisión de entrenamiento: {:.4f}".format(precision))
print("Memoria de entrenamiento: {:.4f}".format(recall))

#Conjunto de evaluación
print()
print("Datos sobre la evaluación")
loss, accuracy,f1_score, precision, recall = model.evaluate(dataTest, targetTest, verbose=False)
print("Exactitud de evaluación: {:.4f}".format(accuracy))
print("F1 de evaluación: {:.4f}".format(f1_score))
print("Precisión de evaluación: {:.4f}".format(precision))
print("Memoria de evaluación: {:.4f}".format(recall))

Datos sobre el entrenamiento
Exactitud de entrenamiento: 0.9968
F1 de entrenamiento: 0.9967
Precisión de entrenamiento: 0.9939
Memoria de entrenamiento: 0.9997

Datos sobre la evaluación
Exactitud de evaluación: 0.9967
F1 de evaluación: 0.9967
Precisión de evaluación: 0.9939
Memoria de evaluación: 0.9997
